# Backend : iteraction with user

In [1]:
# import
import os
import folium
import pyproj
import pandas as pd
import geopandas as gpd
import plotly_express as px
from datetime import datetime, date
import openrouteservice as ors
import matplotlib.pyplot as plt
from shapely.ops import transform
from IPython.display import display
from shapely.geometry import Point, LineString, Polygon

/Users/mac/Desktop/2R/env/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.0-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


### User input coordinates : addresses later.

In [2]:
# input coordinates
coordinates = [[2.3668617010116577,48.86100925394748 ], [2.4125343561172485, 48.83392954811057]]

In [3]:
# openrouteservices 
client = ors.Client(key='5b3ce3597851110001cf62485169f9e2d05c419eaf0607fe2b5b0bfa')

#requete openrouteservice
route = client.directions(
    coordinates=coordinates,
    profile='cycling-regular',
    format ='geojson',
    units ='km'
)

route

{'type': 'FeatureCollection',
 'features': [{'bbox': [2.364594, 48.833494, 2.412849, 48.863729],
   'type': 'Feature',
   'properties': {'segments': [{'distance': 5.578,
      'duration': 1115.6,
      'steps': [{'distance': 0.17,
        'duration': 33.9,
        'type': 11,
        'instruction': 'Head west on Rue du Pont aux Choux',
        'name': 'Rue du Pont aux Choux',
        'way_points': [0, 2]},
       {'distance': 0.018,
        'duration': 3.6,
        'type': 1,
        'instruction': 'Turn right onto Rue de Turenne',
        'name': 'Rue de Turenne',
        'way_points': [2, 4]},
       {'distance': 0.47,
        'duration': 93.9,
        'type': 1,
        'instruction': 'Turn right onto Rue Commines',
        'name': 'Rue Commines',
        'way_points': [4, 22]},
       {'distance': 0.154,
        'duration': 30.8,
        'type': 1,
        'instruction': 'Turn right onto Boulevard Voltaire',
        'name': 'Boulevard Voltaire',
        'way_points': [22, 25]},
   

In [4]:
# get clusters
cluster_polygons_file = "../model/model_cluster/cluster_polygons.geojson"
gdf_cluster = gpd.read_file(cluster_polygons_file)
display(gdf_cluster.head(2))

# clusters with buffer
orig_crs = 4326
fr_crs = 2154
gdf_cluster = gdf_cluster.to_crs(epsg=fr_crs)

# crs
buffer_m = 15
gdf_cluster['geometry'] = gdf_cluster.geometry.buffer(buffer_m)
#gdf_cluster = gdf_cluster.buffer(buffer_m)

# set crs
gdf_cluster = gdf_cluster.to_crs(epsg=orig_crs)

type(gdf_cluster)
gdf_cluster

,cluster_index,n_accidents,geometry
0,0,8,"POLYGON ((2.41149 48.84703, 2.41106 48.84705, ..."
1,1,8,"POLYGON ((2.36897 48.85400, 2.36893 48.85403, ..."


,cluster_index,n_accidents,geometry
0,0,8,"POLYGON ((2.41169 48.84707, 2.41170 48.84705, ..."
1,1,8,"POLYGON ((2.36917 48.85403, 2.36917 48.85401, ..."
2,2,7,"POLYGON ((2.37251 48.84846, 2.37249 48.84846, ..."
3,3,5,"POLYGON ((2.31386 48.86467, 2.31386 48.86465, ..."
4,4,5,"POLYGON ((2.29898 48.86840, 2.29897 48.86839, ..."
...,...,...,...
139,139,5,"POLYGON ((2.37868 48.85880, 2.37869 48.85879, ..."
140,140,6,"POLYGON ((2.36493 48.85404, 2.36495 48.85404, ..."
141,141,5,"POLYGON ((2.35992 48.87225, 2.35993 48.87223, ..."
142,142,6,"POLYGON ((2.37064 48.89819, 2.37062 48.89819, ..."


In [5]:
# user route to df
user_line_coords = route['features'][0]['geometry']['coordinates']
user_line_df = pd.DataFrame(user_line_coords,
                            columns=["lon", "lat"])

# df to gdf
user_line_gdf = gpd.GeoDataFrame(user_line_df, 
                    geometry=gpd.points_from_xy(user_line_df.lon, user_line_df.lat),
                    crs="epsg:4326")

display(user_line_gdf.sample(2))

,lon,lat,geometry
39,2.375197,48.860647,POINT (2.37520 48.86065)
110,2.395631,48.844942,POINT (2.39563 48.84494)


In [6]:
# intersect : spatial join
cluster_on_route = gpd.sjoin(gdf_cluster, user_line_gdf)[gdf_cluster.columns]
display(cluster_on_route.sample(2))

,cluster_index,n_accidents,geometry
139,139,5,"POLYGON ((2.37868 48.85880, 2.37869 48.85879, ..."
18,18,5,"POLYGON ((2.40662 48.83522, 2.40660 48.83522, ..."


In [7]:
# localisation Paris
m = folium.Map(location=[48.862, 2.346], tiles='cartodbpositron', zoom_start=12)

# # cluster on map
# gdf = gdf_cluster
# for _, r in gdf.iterrows():
#     # Without simplifying the representation of each borough,
#     # the map might not be displayed
#     sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
#     geo_j = sim_geo.to_json()
#     geo_j = folium.GeoJson(data=geo_j,
#                            style_function=lambda x: 
#                            {'fillColor': '#669bbc',
#                             'color': '#669bbc'}
#     )
    
#     folium.Popup(r['cluster_index']).add_to(geo_j)
#     geo_j.add_to(m)
    
# cluster intersection map
gdf = cluster_on_route
for _, r in gdf.iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: 
                           {'fillColor': '#eb353c',
                            'color': '#eb353c',
                            'weight':5,
                            'stroke':False}
    )
    
    folium.Popup(r['cluster_index']).add_to(geo_j)
    geo_j.add_to(m)
    
# add user itinerary polyline
folium.PolyLine(locations=[list(reversed(coord)) 
                           for coord in user_line_coords],
                           color = '#84dcc6',
                           smooth_factor=0.9,
                           weight = 1.5

).add_to(m)

In [8]:
m

In [9]:
# export map
m.save("./templates/map.html")

In [10]:
## GEOFENCING
# add time field in user route 
pip_mask_geofence = user_line_gdf.intersects(gdf_cluster.unary_union)

# geofence column
user_line_gdf["geofence"] = pip_mask_geofence

# add time
number_of_points = user_line_gdf.shape[0]
start_date = datetime(2021, 11, 22, 12, 10)
date_range = pd.date_range(start=start_date, periods=number_of_points) #, freq="min"

# time column
user_line_gdf["time"] = date_range

# transform time format to string for geofencing animation
user_line_gdf["time"] = user_line_gdf["time"].apply(lambda x: x.strftime('%Y-%m-%d'))


# replace gepfencing by In and Out
user_line_gdf["geofence"] = user_line_gdf["geofence"].replace({True: "In", False: "Out"})

user_line_gdf


,lon,lat,geometry,geofence,time
0,2.366860,48.861006,POINT (2.36686 48.86101),Out,2021-11-22
1,2.364678,48.861313,POINT (2.36468 48.86131),Out,2021-11-23
2,2.364594,48.861326,POINT (2.36459 48.86133),Out,2021-11-24
3,2.364598,48.861389,POINT (2.36460 48.86139),Out,2021-11-25
4,2.364603,48.861487,POINT (2.36460 48.86149),Out,2021-11-26
...,...,...,...,...,...
190,2.412759,48.833494,POINT (2.41276 48.83349),Out,2022-05-31
191,2.412849,48.833551,POINT (2.41285 48.83355),Out,2022-06-01
192,2.412773,48.833630,POINT (2.41277 48.83363),Out,2022-06-02
193,2.412632,48.833771,POINT (2.41263 48.83377),Out,2022-06-03


In [11]:
# geofencing
px.set_mapbox_access_token("pk.eyJ1Ijoic2hha2Fzb20iLCJhIjoiY2plMWg1NGFpMXZ5NjJxbjhlM2ttN3AwbiJ9.RtGYHmreKiyBfHuElgYq_w")
fig = px.scatter_mapbox(
    user_line_gdf, 
    lat="lat", 
    lon="lon" ,
    size_max=6, 
    zoom=10, width=700, height=400)
#fig.show(renderer="iframe")
fig.show()

In [12]:
px.set_mapbox_access_token("pk.eyJ1Ijoic2hha2Fzb20iLCJhIjoiY2plMWg1NGFpMXZ5NjJxbjhlM2ttN3AwbiJ9.RtGYHmreKiyBfHuElgYq_w")
fig_3 = px.scatter_mapbox(
    user_line_gdf, 
    lat="lat", 
    lon="lon", 
    color="geofence", 
    #size="track_id", 
    animation_frame="time", 
    size_max=100, 
    zoom=12, 
    width=1200, 
    height=800,
    #mapbox_style="dark"
)
fig_3.show(renderer="iframe")
fig_3.show()